<center>
<img src="../../img/ods_stickers.jpg">
## Открытый курс по машинному обучению
</center>
<center>Автор материала: программист-исследователь Mail.ru Group, старший преподаватель <br>Факультета Компьютерных Наук ВШЭ Юрий Кашницкий

# <center> Домашнее задание № 8. Часть 2
## <center> Vowpal Wabbit в задаче классификации тегов вопросов на Stackoverflow

## План 2 части домашнего задания
    2.1. Введение
    2.2. Описание данных
    2.3. Предобработка данных
    2.4. Обучение и проверка моделей
    2.5. Заключение

### 2.1. Введение

В этом задании вы будете делать примерно то же, что я каждую неделю –  в Mail.ru Group: обучать модели на выборке в несколько гигабайт. Задание можно выполнить и на Windows с Python, но я рекомендую поработать под \*NIX-системой (например, через Docker) и активно использовать язык bash.
Немного снобизма (простите, но правда): если вы захотите работать в лучших компаниях мира в области ML, вам все равно понадобится опыт работы с bash под UNIX.

[Веб-форма](https://goo.gl/forms/z8zENbMiaEAeB7nG3) для ответов.

Для выполнения задания понадобится установленный Vowpal Wabbit (уже есть в докер-контейнере курса, см. инструкцию в README [репозитория](https://github.com/Yorko/mlcourse_open) нашего курса) и примерно 70 Гб дискового пространства. Я тестировал решение не на каком-то суперкомпе, а на Macbook Pro 2015 (8 ядер, 16 Гб памяти), и самая тяжеловесная модель обучалась около 12 минут, так что задание реально выполнить и с простым железом. Но если вы планируете когда-либо арендовать сервера Amazon, можно попробовать это сделать уже сейчас.

Материалы в помощь:
 - интерактивный [тьюториал](https://www.codecademy.com/en/courses/learn-the-command-line/lessons/environment/exercises/bash-profile) CodeAcademy по утилитам командной строки UNIX (примерно на час-полтора)
 - [статья](https://habrahabr.ru/post/280562/) про то, как арендовать на Amazon машину (еще раз: это не обязательно для выполнения задания, но будет хорошим опытом, если вы это делаете впервые)

### 2.2. Описание данных

Имеются 10 Гб вопросов со StackOverflow – [скачайте](https://yadi.sk/d/krikdUic3Ggjyf) эту выборку. 

Формат данных простой:<br>
<center>*текст вопроса* (слова через пробел) TAB *теги вопроса* (через пробел)

Здесь TAB – это символ табуляции.
Пример первой записи в выборке:

In [1]:
!head -1 stackoverflow.10kk.tsv

 is there a way to apply a background color through css at the tr level i can apply it at the td level like this my td background color e8e8e8 background e8e8e8 however the background color doesn t seem to get applied when i attempt to apply the background color at the tr level like this my tr background color e8e8e8 background e8e8e8 is there a css trick to making this work or does css not natively support this for some reason 	css css3 css-selectors


Здесь у нас текст вопроса, затем табуляция и теги вопроса: *css, css3* и *css-selectors*. Всего в выборке таких вопросов 10 миллионов. 

In [2]:
%%time
!wc -l stackoverflow.10kk.tsv

10000000 stackoverflow.10kk.tsv
CPU times: user 2.33 s, sys: 240 ms, total: 2.57 s
Wall time: 1min 36s


Обратите внимание на то, что такие данные я уже не хочу загружать в оперативную память и, пока можно, буду пользоваться эффективными утилитами UNIX –  head, tail, wc, cat, cut и прочими.

### 2.3. Предобработка данных

Давайте выберем в наших данных все вопросы с тегами *javascript, java, python, ruby, php, c++, c#, go, scala* и  *swift* и подготовим обучающую выборку в формате Vowpal Wabbit. Будем решать задачу 10-классовой классификации вопросов по перечисленным тегам.

Вообще, как мы видим, у каждого вопроса может быть несколько тегов, но мы упростим себе задачу и будем у каждого вопроса выбирать один из перечисленных тегов либо игнорировать вопрос, если таковых тегов нет. 
Но вообще VW поддерживает multilabel classification (аргумент  --multilabel_oaa).
<br>
<br>
Реализуйте в виде отдельного файла `preprocess.py` код для подготовки данных. Он должен отобрать строки, в которых есть перечисленные теги, и переписать их в отдельный файл в формат Vowpal Wabbit. Детали:
 - скрипт должен работать с аргументами командной строки: с путями к файлам на входе и на выходе
 - строки обрабатываются по одной (можно использовать tqdm для подсчета числа итераций)
 - если табуляций в строке нет или их больше одной, считаем строку поврежденной и пропускаем
 - в противном случае смотрим, сколько в строке тегов из списка *javascript, java, python, ruby, php, c++, c#, go, scala* и  *swift*. Если ровно один, то записываем строку в выходной файл в формате VW: `label | text`, где `label` – число от 1 до 10 (1 - *javascript*, ... 10 – *swift*). Пропускаем те строки, где интересующих тегов больше или меньше одного 
 - из текста вопроса надо выкинуть двоеточия и вертикальные палки, если они есть – в VW это спецсимволы

In [3]:
import os
from tqdm import tqdm
from time import time
import numpy as np
from sklearn.metrics import accuracy_score

Должно получиться вот такое число строк – 4389054. Как видите, 10 Гб у меня обработались примерно за полторы минуты.

In [8]:
%%time
!python preprocessing.py stackoverflow.10kk.tsv stackoverflow.vw answers.txt

4389058
CPU times: user 3.53 s, sys: 380 ms, total: 3.91 s
Wall time: 2min 25s


In [9]:
%%time
!wc -l stackoverflow.vw

4389058 stackoverflow.vw
CPU times: user 1 s, sys: 136 ms, total: 1.14 s
Wall time: 43.4 s


In [10]:
%%time
!wc -l answers.txt

4389058 answers.txt
CPU times: user 16 ms, sys: 0 ns, total: 16 ms
Wall time: 559 ms


Поделите выборку на обучающую, проверочную и тестовую части в равной пропорции - по  1463018 в каждый файл. Перемешивать не надо, первые 1463018 строк должны пойти в обучающую часть `stackoverflow_train.vw`, последние 1463018 – в тестовую `stackoverflow_test.vw`, оставшиеся – в проверочную `stackoverflow_valid.vw`. 

Также сохраните векторы ответов для проверочной и тестовой выборки в отдельные файлы `stackoverflow_valid_labels.txt` и `stackoverflow_test_labels.txt`.

Тут вам помогут утилиты `head`, `tail`, `split`, `cat` и `cut`.

In [33]:
%%time
!split -l 1463020 stackoverflow.vw stackoverflow_

CPU times: user 2.26 s, sys: 280 ms, total: 2.54 s
Wall time: 1min 35s


In [11]:
%%time
!split -l 1463020 answers.txt answers_

CPU times: user 4 ms, sys: 4 ms, total: 8 ms
Wall time: 225 ms


In [12]:
!head -n2 stackoverflow.vw

1 |  i ve got some code in window scroll that checks if an element is visible then triggers another function however only the first section of code is firing both bits of code work in and of themselves if i swap their order whichever is on top fires correctly my code is as follows fn isonscreen function use strict var win window viewport top win scrolltop left win scrollleft bounds this offset viewport right viewport left + win width viewport bottom viewport top + win height bounds right bounds left + this outerwidth bounds bottom bounds top + this outerheight return viewport right lt bounds left viewport left gt bounds right viewport bottom lt bounds top viewport top gt bounds bottom window scroll function use strict var load_more_results ajax load_more_results isonscreen if load_more_results true loadmoreresults var load_more_staff ajax load_more_staff isonscreen if load_more_staff true loadmorestaff what am i doing wrong can you only fire one event from window scroll i assume not 
4

In [13]:
!head -n2 answers.txt

1
4


### 2.4. Обучение и проверка моделей

Обучите Vowpal Wabbit на выборке `stackoverflow_train.vw` 9 раз, перебирая параметры passes (1,3,5), ngram (1,2,3).
Остальные параметры укажите следующие: bit_precision=28 и seed=17. Также скажите VW, что это 10-классовая задача.

Проверяйте долю правильных ответов на выборке `stackoverflow_valid.vw`. Выберите лучшую модель и проверьте качество на выборке `stackoverflow_test.vw`.

In [51]:
%%time
!vw --oaa 10 stackoverflow_train.vw \
-f stackoverflow_model11.vw --passes=1 --ngram=1 --bit_precision=28 --random_seed=17

Generating 1-grams for all namespaces.
final_regressor = stackoverflow_model11.vw
Num weight bits = 28
learning rate = 0.5
initial_t = 0
power_t = 0.5
using no cache
Reading datafile = stackoverflow_train.vw
num sources = 1
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features
0.000000 0.000000            1            1.0        1        1      161
0.500000 1.000000            2            2.0        4        1       68
0.750000 1.000000            4            4.0        7        1       88
0.750000 0.750000            8            8.0        7        1       95
0.812500 0.875000           16           16.0        7        7      209
0.812500 0.812500           32           32.0        7        5      174
0.781250 0.750000           64           64.0        3        3      204
0.671875 0.562500          128          128.0        1        5       29
0.609375 0.546875          256          256.0 

In [52]:
%%time
!vw --oaa 10 stackoverflow_train.vw \
-f stackoverflow_model12.vw --passes=1 --ngram=2 --bit_precision=28 --random_seed=17

Generating 2-grams for all namespaces.
final_regressor = stackoverflow_model12.vw
Num weight bits = 28
learning rate = 0.5
initial_t = 0
power_t = 0.5
using no cache
Reading datafile = stackoverflow_train.vw
num sources = 1
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features
0.000000 0.000000            1            1.0        1        1      320
0.500000 1.000000            2            2.0        4        1      134
0.750000 1.000000            4            4.0        7        1      174
0.750000 0.750000            8            8.0        7        1      188
0.750000 0.750000           16           16.0        7        7      416
0.781250 0.812500           32           32.0        7        2      346
0.765625 0.750000           64           64.0        3        3      406
0.664062 0.562500          128          128.0        1        7       56
0.597656 0.531250          256          256.0 

In [53]:
%%time
!vw --oaa 10 stackoverflow_train.vw \
-f stackoverflow_model13.vw --passes=1 --ngram=3 --bit_precision=28 --random_seed=17

Generating 3-grams for all namespaces.
final_regressor = stackoverflow_model13.vw
Num weight bits = 28
learning rate = 0.5
initial_t = 0
power_t = 0.5
using no cache
Reading datafile = stackoverflow_train.vw
num sources = 1
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features
0.000000 0.000000            1            1.0        1        1      478
0.500000 1.000000            2            2.0        4        1      199
0.750000 1.000000            4            4.0        7        1      259
0.750000 0.750000            8            8.0        7        1      280
0.750000 0.750000           16           16.0        7        7      622
0.781250 0.812500           32           32.0        7        2      517
0.765625 0.750000           64           64.0        3        3      607
0.656250 0.546875          128          128.0        1        7       82
0.605469 0.554688          256          256.0 

In [62]:
%%time
!vw --oaa 10 stackoverflow_train.vw \
-f stackoverflow_model31.vw --passes=3 --ngram=1 --bit_precision=28 --random_seed=17 -c

Generating 1-grams for all namespaces.
final_regressor = stackoverflow_model31.vw
Num weight bits = 28
learning rate = 0.5
initial_t = 0
power_t = 0.5
decay_learning_rate = 1
creating cache_file = stackoverflow_train.vw.cache
Reading datafile = stackoverflow_train.vw
num sources = 1
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features
0.000000 0.000000            1            1.0        1        1      161
0.500000 1.000000            2            2.0        4        1       68
0.750000 1.000000            4            4.0        7        1       88
0.750000 0.750000            8            8.0        7        1       95
0.750000 0.750000           16           16.0        2        7      159
0.781250 0.812500           32           32.0        1        7      404
0.718750 0.656250           64           64.0        7        7      103
0.664062 0.609375          128          128.0        5     

In [63]:
%%time
!vw --oaa 10 stackoverflow_train.vw \
-f stackoverflow_model32.vw --passes=3 --ngram=2 --bit_precision=28 --random_seed=17 -c

Generating 2-grams for all namespaces.
final_regressor = stackoverflow_model32.vw
Num weight bits = 28
learning rate = 0.5
initial_t = 0
power_t = 0.5
decay_learning_rate = 1
using cache_file = stackoverflow_train.vw.cache
ignoring text input in favor of cache input
num sources = 1
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features
0.000000 0.000000            1            1.0        1        1      320
0.500000 1.000000            2            2.0        4        1      134
0.750000 1.000000            4            4.0        7        1      174
0.750000 0.750000            8            8.0        7        1      188
0.687500 0.625000           16           16.0        2        7      316
0.687500 0.687500           32           32.0        1        7      806
0.687500 0.687500           64           64.0        7        7      204
0.601562 0.515625          128          128.0        5      

In [64]:
%%time
!vw --oaa 10 stackoverflow_train.vw \
-f stackoverflow_model33.vw --passes=3 --ngram=3 --bit_precision=28 --random_seed=17 -c

Generating 3-grams for all namespaces.
final_regressor = stackoverflow_model33.vw
Num weight bits = 28
learning rate = 0.5
initial_t = 0
power_t = 0.5
decay_learning_rate = 1
using cache_file = stackoverflow_train.vw.cache
ignoring text input in favor of cache input
num sources = 1
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features
0.000000 0.000000            1            1.0        1        1      478
0.500000 1.000000            2            2.0        4        1      199
0.750000 1.000000            4            4.0        7        1      259
0.750000 0.750000            8            8.0        7        1      280
0.750000 0.750000           16           16.0        2        7      472
0.750000 0.750000           32           32.0        1        7     1207
0.718750 0.687500           64           64.0        7        7      304
0.640625 0.562500          128          128.0        5      

In [65]:
%%time
!vw --oaa 10 stackoverflow_train.vw \
-f stackoverflow_model51.vw --passes=5 --ngram=1 --bit_precision=28 --random_seed=17 -c

Generating 1-grams for all namespaces.
final_regressor = stackoverflow_model51.vw
Num weight bits = 28
learning rate = 0.5
initial_t = 0
power_t = 0.5
decay_learning_rate = 1
using cache_file = stackoverflow_train.vw.cache
ignoring text input in favor of cache input
num sources = 1
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features
0.000000 0.000000            1            1.0        1        1      161
0.500000 1.000000            2            2.0        4        1       68
0.750000 1.000000            4            4.0        7        1       88
0.750000 0.750000            8            8.0        7        1       95
0.750000 0.750000           16           16.0        2        7      159
0.781250 0.812500           32           32.0        1        7      404
0.718750 0.656250           64           64.0        7        7      103
0.664062 0.609375          128          128.0        5      

In [66]:
%%time
!vw --oaa 10 stackoverflow_train.vw \
-f stackoverflow_model52.vw --passes=5 --ngram=2 --bit_precision=28 --random_seed=17 -c

Generating 2-grams for all namespaces.
final_regressor = stackoverflow_model52.vw
Num weight bits = 28
learning rate = 0.5
initial_t = 0
power_t = 0.5
decay_learning_rate = 1
using cache_file = stackoverflow_train.vw.cache
ignoring text input in favor of cache input
num sources = 1
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features
0.000000 0.000000            1            1.0        1        1      320
0.500000 1.000000            2            2.0        4        1      134
0.750000 1.000000            4            4.0        7        1      174
0.750000 0.750000            8            8.0        7        1      188
0.687500 0.625000           16           16.0        2        7      316
0.687500 0.687500           32           32.0        1        7      806
0.687500 0.687500           64           64.0        7        7      204
0.601562 0.515625          128          128.0        5      

In [67]:
%%time
!vw --oaa 10 stackoverflow_train.vw \
-f stackoverflow_model53.vw --passes=5 --ngram=3 --bit_precision=28 --random_seed=17 -c

Generating 3-grams for all namespaces.
final_regressor = stackoverflow_model53.vw
Num weight bits = 28
learning rate = 0.5
initial_t = 0
power_t = 0.5
decay_learning_rate = 1
using cache_file = stackoverflow_train.vw.cache
ignoring text input in favor of cache input
num sources = 1
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features
0.000000 0.000000            1            1.0        1        1      478
0.500000 1.000000            2            2.0        4        1      199
0.750000 1.000000            4            4.0        7        1      259
0.750000 0.750000            8            8.0        7        1      280
0.750000 0.750000           16           16.0        2        7      472
0.750000 0.750000           32           32.0        1        7     1207
0.718750 0.687500           64           64.0        7        7      304
0.640625 0.562500          128          128.0        5      

<font color='red'> Вопрос 1.</font> Какое сочетание параметров дает наибольшую долю правильных ответов на проверочной выборке `stackoverflow_valid.vw`?
- Биграммы и 3 прохода по выборке
- Триграммы и 5 проходов по выборке
- Биграммы и 1 проход по выборке
- Униграммы и 1 проход по выборке

Проверьте лучшую (по доле правильных ответов на валидации) модель на тестовой выборке. 

In [15]:
with open('answers_valid.txt') as ans_file:
    valid_answer = [float(label) 
                             for label in ans_file.readlines()]

In [17]:
%%time
!vw -i stackoverflow_model11.vw -t -d stackoverflow_valid.vw \
-p stackoverflow_valid_pred11.txt --quiet
with open('stackoverflow_valid_pred11.txt') as pred_file:
    valid_pred = [float(label) 
                             for label in pred_file.readlines()]
print(accuracy_score(valid_answer, valid_pred))

0.915148118276
CPU times: user 1.41 s, sys: 136 ms, total: 1.54 s
Wall time: 26.3 s


In [18]:
!vw -i stackoverflow_model12.vw -t -d stackoverflow_valid.vw \
-p stackoverflow_valid_pred12.txt --quiet
with open('stackoverflow_valid_pred12.txt') as pred_file:
    valid_pred = [float(label) 
                             for label in pred_file.readlines()]
print(accuracy_score(valid_answer, valid_pred))

0.931117824773


In [19]:
!vw -i stackoverflow_model32.vw -t -d stackoverflow_valid.vw \
-p stackoverflow_valid_pred32.txt --quiet
with open('stackoverflow_valid_pred32.txt') as pred_file:
    valid_pred = [float(label) 
                             for label in pred_file.readlines()]
print(accuracy_score(valid_answer, valid_pred))

0.927804131181


In [20]:
!vw -i stackoverflow_model53.vw -t -d stackoverflow_valid.vw \
-p stackoverflow_valid_pred53.txt --quiet
with open('stackoverflow_valid_pred53.txt') as pred_file:
    valid_pred = [float(label) 
                             for label in pred_file.readlines()]
print(accuracy_score(valid_answer, valid_pred))

0.92616300529


<font color='red'> Вопрос 2.</font> Как соотносятся доли правильных ответов лучшей (по доле правильных ответов на валидации) модели на проверочной и на тестовой выборках? (здесь % – это процентный пункт, т.е., скажем, снижение с 50% до 40% – это на 10%, а не 20%).
- На тестовой ниже примерно на 2%
- На тестовой ниже примерно на 3%
- Результаты почти одинаковы – отличаются меньше чем на 0.5%

In [25]:
with open('answers_test.txt') as ans_file:
    test_answer = [float(label) 
                             for label in ans_file.readlines()]
!vw -i stackoverflow_model12.vw -t -d stackoverflow_test.vw \
-p stackoverflow_test_pred12.txt --quiet
with open('stackoverflow_test_pred12.txt') as pred_file:
    test_pred = [float(label) 
                             for label in pred_file.readlines()]
print(accuracy_score(test_answer, test_pred))

0.931202486914


Обучите VW с параметрами, подобранными на проверочной выборке, теперь на объединении обучающей и проверочной выборок. Посчитайте долю правильных ответов на тестовой выборке. 

In [29]:
%%time
!cat stackoverflow_train.vw stackoverflow_valid.vw > stackoverflow_big_train.vw

CPU times: user 2 s, sys: 268 ms, total: 2.27 s
Wall time: 1min 25s


In [30]:
%%time
!wc -l stackoverflow_big_train.vw

2926040 stackoverflow_big_train.vw
CPU times: user 776 ms, sys: 88 ms, total: 864 ms
Wall time: 31.7 s


In [31]:
%%time
!vw --oaa 10 stackoverflow_big_train.vw \
-f stackoverflow_big_model12.vw --passes=1 --ngram=2 --bit_precision=28 --random_seed=17

Generating 2-grams for all namespaces.
final_regressor = stackoverflow_big_model12.vw
Num weight bits = 28
learning rate = 0.5
initial_t = 0
power_t = 0.5
using no cache
Reading datafile = stackoverflow_big_train.vw
num sources = 1
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features
0.000000 0.000000            1            1.0        1        1      320
0.500000 1.000000            2            2.0        4        1      134
0.750000 1.000000            4            4.0        7        1      174
0.750000 0.750000            8            8.0        7        1      188
0.750000 0.750000           16           16.0        7        7      416
0.781250 0.812500           32           32.0        7        2      346
0.765625 0.750000           64           64.0        3        3      406
0.664062 0.562500          128          128.0        1        7       56
0.597656 0.531250          256        

In [32]:
%%time
with open('answers_test.txt') as ans_file:
    test_answer = [float(label) 
                             for label in ans_file.readlines()]
!vw -i stackoverflow_big_model12.vw -t -d stackoverflow_test.vw \
-p stackoverflow_test_pred_big.txt --quiet
with open('stackoverflow_test_pred_big.txt') as pred_file:
    test_pred = [float(label) 
                             for label in pred_file.readlines()]
print(accuracy_score(test_answer, test_pred))

0.935183982699
CPU times: user 4.85 s, sys: 644 ms, total: 5.5 s
Wall time: 2min 42s


<font color='red'> Вопрос 3.</font> На сколько процентных пунктов повысилась доля правильных ответов модели после обучения на вдвое большей выборке (обучающая `stackoverflow_train.vw` + проверочная `stackoverflow_valid.vw`) по сравнению с моделью, обученной только на `stackoverflow_train.vw`?
 - 0.1%
 - 0.4%
 - 0.8%
 - 1.2%

### 2.5. Заключение

В этом задании мы только познакомились с Vowpal Wabbit. Что еще можно попробовать:
 - Классификация с несколькими ответами (multilabel classification, аргумент  `multilabel_oaa`) – формат данных тут как раз под такую задачу
 - Настройка параметров VW с hyperopt, авторы библиотеки утверждают, что качество должно сильно зависеть от параметров изменения шага градиентного спуска (`initial_t` и `power_t`). Также можно потестировать разные функции потерь – обучать логистическую регресиию или линейный SVM
 - Познакомиться с факторизационными машинами и их реализацией в VW (аргумент `lrq`)